In [1]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv, max_pool, max_pool_x, graclus, global_mean_pool, GCNConv,  global_mean_pool, SAGEConv, voxel_grid, SplineConv, GATv2Conv
import torch.nn.functional as F
import torch_scatter

class EventConv_mean_min_max_var(MessagePassing):

    def __init__(self, in_channels, out_channels):
        super(EventConv_mean_min_max_var, self).__init__()
        self.mlp = Seq(
            Linear(out_channels, out_channels), 
            ReLU(), 
            Linear(out_channels, out_channels)
        )

    def forward(self, x, edge_index):
        sara = self.propagate(edge_index, x=x)
        return sara

    def aggregate(self, inputs, index):
        sums = torch_scatter.scatter_add(inputs, index, dim=0)
        maxs = torch_scatter.scatter_max(inputs, index, dim=0)[0]
        means = torch_scatter.scatter_mean(inputs, index, dim=0)
        var = torch.relu(
            torch_scatter.scatter_mean(
                inputs ** 2,
                index,
                dim=0) -
            means ** 2)

        aggrs = torch.hstack((sums, maxs, means, var))
        return self.mlp(aggrs)
    
    
class NetConnect_3e_model3(torch.nn.Module):
    def __init__(self):
        super(NetConnect_3e_model3, self).__init__()
        #self.conv1 = SplineConv(1, 64, dim=3, kernel_size=4)
        ####print("Kholous", len(data))

        in_dim = 3
        hidden_dim = in_dim*4
        self.edge_conv1 = EventConv_mean_min_max_var(in_dim, hidden_dim)

        in_dim2 = in_dim + hidden_dim
        hidden_dim2 = in_dim2 * 4
        ##print(in_dim2, hidden_dim2)
        self.edge_conv1_2L = EventConv_mean_min_max_var(in_dim2, hidden_dim2)

        in_dim3 = hidden_dim2 + in_dim2 
        hidden_dim3 = in_dim3 * 4 
        ##print(in_dim3, hidden_dim3)

        self.edge_conv1_3L = EventConv_mean_min_max_var(in_dim3, hidden_dim3)
        n_heads1 = 8
        self.conv0 = GATConv(hidden_dim3 + in_dim3, 64, n_heads1)
        self.bn0 = torch.nn.BatchNorm1d(64*n_heads1)

        self.conv1 = GATConv(64*n_heads1, 16, n_heads1)
        self.bn1 = torch.nn.BatchNorm1d(16*n_heads1)
        self.conv2 = GATConv(16*n_heads1, 16)
        self.bn2 = torch.nn.BatchNorm1d(16)
        
        self.fc1 = torch.nn.Linear(16 * 64, 128)
        self.fc2 = torch.nn.Linear(128, 2)


    def forward(self, data):

        source_data = data.x.clone().detach()
        data.x = self.edge_conv1(source_data, data.edge_index.clone().detach())
        data.x = torch.sigmoid(data.x)
        part1 = data.x
        ##print('part1', part1.shape)
        source_data_2L = torch.cat((source_data, part1), dim=1)
    
        ##print('source_data_2L', source_data_2L.shape)

        data.x = self.edge_conv1_2L(source_data_2L, data.edge_index.clone().detach())
        data.x = torch.sigmoid(data.x)
        part1_2L = data.x
        ##print('part1_2L', part1_2L.shape)
        source_data_3L = torch.cat((source_data_2L, part1_2L), dim=1)
        ##print('source_data_3L', source_data_3L.shape)
        data.x = self.edge_conv1_3L(
            source_data_3L, data.edge_index.clone().detach())
        data.x = torch.sigmoid(data.x)
        part1_3L = data.x
        ##print('part1_3L', part1_3L.shape)
        source_data_4L = torch.cat((source_data_3L, part1_3L), dim=1).to('cuda')
        ##print('source_data_4L', source_data_4L.shape)
                
        data.x = data.x.to('cuda')
        data.edge_index = data.edge_index.to('cuda')
        data.x = torch.sigmoid(self.conv0(source_data_4L, data.edge_index, data.edge_attr))
    
        data.x = self.bn0(data.x)
        data.x = torch.sigmoid(self.conv1(data.x, data.edge_index, data.edge_attr))
        data.x = self.bn1(data.x)
        data.x = torch.sigmoid(self.conv2(data.x, data.edge_index, data.edge_attr))
        data.x = self.bn2(data.x)
        
        cluster = voxel_grid(data.pos, batch=data.batch, size=0.25)
        x,_ = max_pool_x(cluster, data.x, batch=data.batch, size=64)
        
        x = x.view(-1, self.fc1.weight.size(1))
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training, p=0.7)
        x = self.fc2(x)
        
        return x  # data.x

In [2]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_geometric.utils.subgraph import subgraph
def dropout_node(edge_index, p= 0.5,
                 num_nodes = None,
                 training = True):
    r"""Randomly drops nodes from the adjacency matrix
    :obj:`edge_index` with probability :obj:`p` using samples from
    a Bernoulli distribution.

    The method returns (1) the retained :obj:`edge_index`, (2) the edge mask
    indicating which edges were retained. (3) the node mask indicating
    which nodes were retained.

    Args:
        edge_index (LongTensor): The edge indices.
        p (float, optional): Dropout probability. (default: :obj:`0.5`)
        num_nodes (int, optional): The number of nodes, *i.e.*
            :obj:`max_val + 1` of :attr:`edge_index`. (default: :obj:`None`)
        training (bool, optional): If set to :obj:`False`, this operation is a
            no-op. (default: :obj:`True`)

    :rtype: (:class:`LongTensor`, :class:`BoolTensor`, :class:`BoolTensor`)

    Examples:

        >>> edge_index = torch.tensor([[0, 1, 1, 2, 2, 3],
        ...                            [1, 0, 2, 1, 3, 2]])
        >>> edge_index, edge_mask, node_mask = dropout_node(edge_index)
        >>> edge_index
        tensor([[0, 1],
                [1, 0]])
        >>> edge_mask
        tensor([ True,  True, False, False, False, False])
        >>> node_mask
        tensor([ True,  True, False, False])
    """
    if p < 0. or p > 1.:
        raise ValueError(f'Dropout probability has to be between 0 and 1 '
                         f'(got {p}')


    if not training or p == 0.0:
        node_mask = edge_index.new_ones(num_nodes, dtype=torch.bool)
        edge_mask = edge_index.new_ones(edge_index.size(1), dtype=torch.bool)
        return edge_index, edge_mask, node_mask

    prob = torch.rand(num_nodes, device=edge_index.device)
    node_mask = prob > p
    edge_index, _, edge_mask = subgraph(node_mask, edge_index,
                                        num_nodes=num_nodes,
                                        return_edge_mask=True)
    return edge_index, edge_mask, node_mask

In [4]:
model = NetConnect_3e_model3().cuda()
model

NetConnect_3e_model3(
  (edge_conv1): EventConv_mean_min_max_var()
  (edge_conv1_2L): EventConv_mean_min_max_var()
  (edge_conv1_3L): EventConv_mean_min_max_var()
  (conv0): GATConv(375, 64, heads=8)
  (bn0): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): GATConv(512, 16, heads=8)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GATConv(128, 16, heads=1)
  (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [6]:
tm.train_data.get(1)

Data(x=[1876, 4], edge_index=[2, 60032], y=[1, 2], pos=[1876, 3], edge_attr=[60032, 3])

In [8]:
model(tm.train_data.get(1).cuda())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1876x16 and 12x12)

In [12]:
tm.val_data

TactileDataset(129)

In [10]:
#!rm -rf /home/hussain/tactile/data/contact_extraction7/*/processed/

/bin/bash: /home/hussain/miniconda3/envs/tactile/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [19]:
np.sqrt(0.1)

NameError: name 'np' is not defined

In [21]:
from imports.TrainModel import TrainModel
#!rm -rf ../data/contact_extraction7/{train,test,val}/processed/
tm = TrainModel(
    '/home/hussain/tactile/data/contact_extraction7/', 
    model, 
    features='coords',
    lr=0.01,
    #weight_decay=0.00005, 
    n_epochs=500,
    augment=True,
    batch=2
)
tm.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(tm.optimizer, factor=0.31622776601)
tm.train()

training:   0%|          | 0/500 [00:00<?, ?epoch/s]

  0%|          | 0/772 [00:00<?, ?batch/s]

logging


  0%|          | 0/772 [00:00<?, ?batch/s]

logging


...

logging


  0%|          | 0/772 [00:00<?, ?batch/s]

logging


  0%|          | 0/772 [00:00<?, ?batch/s]

KeyboardInterrupt: 